Needed dependencies:

  * q-alchemy-sdk-py
  * jupyter
  * scikit-learn

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
import pennylane as qml
from pennylane.tape import QuantumTape

from q_alchemy.parser.qasm_pennylane import from_qasm
from q_alchemy.initialize import q_alchemy_as_qasm

In [2]:
mnist = fetch_openml('mnist_784', version=1, parser="auto")

zero: np.ndarray = mnist.data[mnist.target == "0"].iloc[0].to_numpy()
filler = np.empty(2 ** 10 - zero.shape[0])
filler.fill(0)

zero = np.hstack([zero, filler])
zero = zero / np.linalg.norm(zero)

# Qiskit

In [3]:
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import Initialize
from q_alchemy.qiskit_integration import QAlchemyInitialize

qc_qy = QAlchemyInitialize(zero.tolist(), opt_params={"api_key": "qal_SuEckTkN_BdtHczpq7ShJ7kTrjFo1BKtu"})
qc_qk = Initialize(zero)

state_qy = Statevector(qc_qy.definition).data
state_qk = Statevector(qc_qk.definition).data

In [4]:
abs(np.vdot(zero, state_qk))**2, abs(np.vdot(zero, state_qy))**2

(1.0000000000008722, 1.0000000000005098)

# PennyLane

In [5]:
dev = qml.device('default.qubit')

@qml.qnode(dev)
def circuit(state):
    qml.MottonenStatePreparation(state_vector=state, wires=range(10))
    return qml.state()

with QuantumTape() as tape:
    circuit(zero)
    
tape.specs["resources"]

wires: 10
gates: 2045
depth: 2036
shots: Shots(total=None)
gate_types:
{'RY': 1023, 'CNOT': 1022}
gate_sizes:
{1: 1023, 2: 1022}


In [9]:
from q_alchemy.pennylane_integration import QAlchemyStatePreparation 

dev = qml.device('default.qubit')

@qml.qnode(dev)
def circuit_qy(state):
    QAlchemyStatePreparation(state_vector=state, wires=range(10), max_fidelity_loss=0.0)
    return qml.state()

with QuantumTape() as tape:
    circuit_qy(zero)
    
tape.expand().specs["resources"]

wires: 10
gates: 6472
depth: 2231
shots: Shots(total=None)
gate_types:
{'Rot': 1853, 'PhaseShift': 3706, 'CNOT': 913}
gate_sizes:
{1: 5559, 2: 913}


In [10]:
qy_state = circuit_qy(zero)
qml_state = circuit(zero)

In [11]:
abs(np.vdot(zero, qml_state))**2, abs(np.vdot(zero, qy_state))**2

(0.9999999999999978, 1.0000000000000422)

# Pure

In [3]:
qasm, summary = q_alchemy_as_qasm(zero, max_fidelity_loss=0.5, api_key="qal_SuEckTkN_BdtHczpq7ShJ7kTrjFo1BKtu", return_summary=True, basis_gates=["u1", "u2", "u3", "cx"])
len(qasm.split("\n"))

219

In [4]:
summary

{'fidelity_loss': 0.49188161632960237,
 'min_fidelity': 0.5,
 'circuit_depth': 31,
 'circuit_ops': {'u3': 159, 'cx': 57},
 'n_qubits': 10,
 'status': 'OK'}

In [7]:
with QuantumTape() as qasm_2_qml_tape:
    from_qasm(qasm)
    
qasm_2_qml_tape.specs["resources"]

wires: 10
gates: 216
depth: 31
shots: Shots(total=None)
gate_types:
{'U3': 159, 'CNOT': 57}
gate_sizes:
{1: 159, 2: 57}
